In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)

await asyncio.gather(attcs.start_task, latiss.start_task)

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'

In [ ]:
await latiss.take_bias(nbias=10, group_id=group_id)

In [ ]:
await latiss.take_darks(exptime=5.0, ndarks=10, group_id=group_id)

In [ ]:
#await attcs.prepare_for_flatfield()
# The following approximates the above because some things don't work well
await attcs.open_m1_cover()
await attcs.point_azel(target_name="FlatField position",
                       az=attcs.tel_flat_az,
                       el=attcs.tel_flat_el,
                       wait_dome=False)
await attcs.slew_dome_to(attcs.dome_flat_az)

In [ ]:
await latiss.take_flats(exptime=2.0, nflats=10, group_id=group_id)